In [204]:
import yfinance as yf

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import requests
from tqdm import tqdm

import talib as ta

### Question 1: Withdrawn IPOs by Company Type
What is the total withdrawn IPO value (in $ millions) for the company class with the highest total withdrawal value?

From the withdrawn IPO list [stockanalysis.com/ipos/withdrawn](stockanalysis.com/ipos/withdrawn), collect and process the data to find out which company type saw the most withdrawn IPO value.

#### Definition of terms
* **IPO withdrawal:** In the world of stock analytics, this refers to a situation where a company that has filed to go public (i.e., launched an Initial Public Offering) decides to cancel or postpone the offering before its shares are officially listed and traded on a stock exchange.

In [46]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/withdrawn/"
response = requests.get(url, headers=headers)

ipo_withdraw = pd.read_html(response.text)

C:\Users\DELL\AppData\Local\Temp\ipykernel_16984\3236582231.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_withdraw = pd.read_html(response.text)


In [47]:
ipo_withdraw_dfs = ipo_withdraw[0]
ipo_withdraw_dfs.head(2)

,Symbol,Company Name,Price Range,Shares Offered
0,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
1,AURN,"Aurion Biotech, Inc.",-,-


In [303]:
ipo_withdraw_dfs['Company Name'].nunique(), ipo_withdraw_dfs.shape

(99, (99, 6))

In [48]:
ipo_withdraw_dfs.loc[0]

Symbol                              UNFL
Company Name      Unifoil Holdings, Inc.
Price Range                $3.00 - $4.00
Shares Offered                   2000000
Name: 0, dtype: object

In [49]:
ipo_withdraw_dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Symbol          99 non-null     object
 1   Company Name    99 non-null     object
 2   Price Range     99 non-null     object
 3   Shares Offered  99 non-null     object
dtypes: object(4)
memory usage: 3.2+ KB


In [58]:
# IPO value will the thw average of price range * shared offer
def split_price_range(x):
    if x == '-' or pd.isna(x):
        return 0.0
    
    x = x.replace('$', '').replace(' ', '')
    if '-' in x:
        low, high = map(float, x_lst)
        return (low + high) / 2
    
    return float(x)

In [55]:
ipo_withdraw_dfs['Price Per Share'] = ipo_withdraw_dfs['Price Range'].apply(split_price_range)
ipo_withdraw_dfs['Shares Offered'] = ipo_withdraw_dfs['Shares Offered'].apply(split_price_range)
ipo_withdraw_dfs['IPO Price'] = ipo_withdraw_dfs['Price Per Share'] * ipo_withdraw_dfs['Shares Offered']

In [79]:
ipo_price_order = ipo_withdraw_dfs.sort_values('IPO Price', ascending=False).reset_index(drop=True)
ipo_price_order['IPO Price'] = round(ipo_price_order['IPO Price'] / 1000000, 2).astype(str) + 'M'

max_ipo_price = ipo_price_order.loc[0]
max_ipo_price

Symbol                         NVL
Company Name          Novelis Inc.
Price Range        $18.00 - $21.00
Shares Offered          45000000.0
Price Per Share               19.5
IPO Price                   877.5M
Name: 0, dtype: object

### Question 2: Median Sharpe Ratio for 2024 IPOs (First 5 Months)
What is the median Sharpe ratio (as of 6 June 2025) for companies that went public in the first 5 months of 2024?

Data Source: Using the same approach as in Question 1, download the IPOs in 2024 from:
https://stockanalysis.com/ipos/2024/

#### Definition of terms
* **Volatility:** this is a measure of how much a stock’s price moves up and down over time.
* **Sharpe Ratio:** this is a financial metric used to evaluate the risk-adjusted return of an investment. It tells you how much excess return you’re getting for the extra volatility you endure by holding a riskier asset. Example: if rist rate is 4.5%, positive Sharpe means growth exceeding the risk-free rate of 4.5%.

In [109]:
url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)

ipo_stock = pd.read_html(response.text)

C:\Users\DELL\AppData\Local\Temp\ipykernel_16984\2197623396.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_stock = pd.read_html(response.text)


In [110]:
ipo_stock_df = ipo_stock[0]

ipo_stock_df.head(2)

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"Dec 31, 2024",ONEG,OneConstruction Group Limited,$4.00,$3.55,-12.25%
1,"Dec 27, 2024",PHH,"Park Ha Biological Technology Co., Ltd.",$4.00,$18.32,346.00%


In [111]:
ipo_stock_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      225 non-null    object
 1   Symbol        225 non-null    object
 2   Company Name  225 non-null    object
 3   IPO Price     225 non-null    object
 4   Current       225 non-null    object
 5   Return        225 non-null    object
dtypes: object(6)
memory usage: 10.7+ KB


In [119]:
# change data type
# remove dollar sign
def clean_special_chars(cols):
    for c in cols:
        ipo_stock_df[c] = ipo_stock_df[c].apply(lambda x: 0 if x == '-' else x)
        if c == 'Return':
            ipo_stock_df[c] = ipo_stock_df[c].str.replace('%', '')
        else:
            ipo_stock_df[c] = ipo_stock_df[c].str.replace('$', '')
        

def change_type_float(cols):
    for c in cols:
        ipo_stock_df[c] = ipo_stock_df[c].astype(float)

def change_type_date(col):
    ipo_stock_df[col] = pd.to_datetime(ipo_stock_df[col])


In [120]:
# clean cols
cols = ['IPO Price', 'Current', 'Return']
clean_special_chars(cols)
change_type_float(cols)
change_type_date('IPO Date')

In [124]:
trim_ipo_stock_df = ipo_stock_df[ipo_stock_df['IPO Date'] < '2024-06-01'].reset_index(drop=True)
trim_ipo_stock_df.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2024-05-31,NAKA,"Kindly MD, Inc.",NaN,14.18,NaN
1,2024-05-23,BOW,Bowhead Specialty Holdings Inc.,17.00,35.09,114.06
2,2024-05-17,HDL,Super Hi International Holding Ltd.,19.56,20.41,4.50
3,2024-05-17,RFAI,RF Acquisition Corp II,10.00,10.50,5.10
4,2024-05-15,JDZG,JIADE Limited,4.00,0.27,-92.63


In [163]:
trim_ipo_stock_df.shape

(77, 6)

**Daily Stock Data**

In [166]:
risk_free_rate = 0.045

first_ticker = trim_ipo_stock_df.Symbol[0]

stock_df = yf.download(tickers = first_ticker,
                     period = "max",
                     interval = "1d",
                     auto_adjust=False)
stock_df.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
stock_df['Ticker'] = first_ticker
stock_df['Growth 252'] = stock_df['Close'] / stock_df['Close'].shift(252)

# how much stock prices move up and down
stock_df['Returns'] = stock_df['Close'].pct_change() # daily returns
# I could have calculated volatility on daily returns if I want
stock_df['Volatility (Yearly)']  = stock_df['Close'].rolling(30).std() * 252**0.5  


[*********************100%***********************]  1 of 1 completed


In [168]:
stock_df = stock_df.reset_index()
stock_df.tail()

,Date,Adj Close,Close,High,Low,Open,Volume,Ticker,Growth 252,Returns,Volatility (Yearly)
251,2025-06-03,18.360001,18.360001,22.003000,16.700001,21.500000,1336500,NAKA,NaN,-0.137623,143.730280
252,2025-06-04,17.129999,17.129999,18.860001,16.510000,18.129999,770800,NAKA,5.672185,-0.066994,142.127837
253,2025-06-05,16.020000,16.020000,17.874001,15.430000,17.150000,672700,NAKA,6.022556,-0.064799,139.767994
254,2025-06-06,15.880000,15.880000,17.200001,15.550000,16.260000,431200,NAKA,5.438356,-0.008739,136.860031
255,2025-06-09,13.350000,13.350000,15.920000,12.785000,15.900000,972800,NAKA,4.890110,-0.159320,133.213858


In [170]:
for ticker in tqdm(trim_ipo_stock_df.Symbol[1:]):
    next_stock_df = yf.download(tickers = ticker,
                     period = "max",
                     interval = "1d",
                     auto_adjust=False)
    next_stock_df.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
    next_stock_df['Ticker'] = ticker
    next_stock_df['Growth 252'] = next_stock_df['Close'] / next_stock_df['Close'].shift(252)
    # how much stock prices move up and down
    next_stock_df['Returns'] = next_stock_df['Close'].pct_change() # daily returns
    # I could have calculated volatility on daily returns if I want
    next_stock_df['Volatility (Yearly)']  = next_stock_df['Close'].rolling(30).std() * 252**0.5  

    next_stock_df = next_stock_df.reset_index()
    stock_df = pd.concat([stock_df, next_stock_df], ignore_index=True)

[*********************100%***********************]  1 of 1 completed                                                       | 0/76 [00:00<?, ?it/s]
[*********************100%***********************]  1 of 1 completed                                               | 1/76 [00:00<00:55,  1.35it/s]
[*********************100%***********************]  1 of 1 completed                                               | 2/76 [00:01<00:53,  1.40it/s]
[*********************100%***********************]  1 of 1 completed                                               | 3/76 [00:02<00:52,  1.38it/s]
[*********************100%***********************]  1 of 1 completed                                               | 4/76 [00:02<00:54,  1.31it/s]
[*********************100%***********************]  1 of 1 completed                                               | 5/76 [00:03<00:56,  1.27it/s]
[*********************100%***********************]  1 of 1 completed                                               | 6

In [171]:
stock_df.tail()

,Date,Adj Close,Close,High,Low,Open,Volume,Ticker,Growth 252,Returns,Volatility (Yearly)
23259,2025-06-03,3.41,3.41,3.530,3.160,3.36,67200,ROMA,5.262346,0.014881,10.986046
23260,2025-06-04,3.76,3.76,3.850,3.457,3.50,271100,ROMA,5.829458,0.102639,11.789832
23261,2025-06-05,3.57,3.57,4.135,3.360,3.74,264600,ROMA,6.144579,-0.050532,12.173744
23262,2025-06-06,3.70,3.70,3.950,3.630,3.65,84400,ROMA,6.156406,0.036415,12.553202
23263,2025-06-09,2.86,2.86,3.955,2.600,3.75,381700,ROMA,4.627832,-0.227027,12.238182


In [172]:
stock_df['Sharpe'] = (stock_df['Growth 252'] - risk_free_rate) / stock_df['Volatility (Yearly)']

**Explore trading on `2025-06-06`**

In [173]:
stocke_df_20250606 = stock_df[stock_df['Date'] == '2025-06-06']
stocke_df_20250606[['Growth 252', 'Sharpe']].describe()

,Growth 252,Sharpe
count,73.000000,73.000000
mean,1.223993,0.297260
std,1.478959,0.523296
min,0.024970,-0.079677
25%,0.293510,0.040265
50%,0.763188,0.082241
75%,1.446667,0.331967
max,8.097413,2.835668


* `73` out of `77` companies traded on `2025-06-06`.
* Median of `Growth 252` is `0.76` (indicating a `24%` decline) while mean is `1.22` showing bias towards high growth companies (outliers), pushing the average upward.
* The dedian of Sharp Ratio is `0.08`. A positive Sharpe Ratio median for these `73` stocks growth exceeding the risk-free rate of `4.5%`.

Now lets observer the top 10 companies by growth_252d and by Sharpe

In [179]:
stocke_df_20250606.sort_values(by='Growth 252', ascending=False).reset_index(drop=True).loc[:10]

,Date,Adj Close,Close,High,Low,Open,Volume,Ticker,Growth 252,Returns,Volatility (Yearly),Sharpe
0,2025-06-06,5.320000,5.320000,5.355000,5.289000,5.310000,7700,JL,8.097413,0.009488,14.221305,0.566222
1,2025-06-06,3.700000,3.700000,3.950000,3.630000,3.650000,84400,ROMA,6.156406,0.036415,12.553202,0.486840
2,2025-06-06,15.880000,15.880000,17.200001,15.550000,16.260000,431200,NAKA,5.438356,-0.008739,136.860031,0.039408
3,2025-06-06,7.420000,7.420000,8.650000,6.610000,6.810000,10186200,UMAC,4.966533,0.138037,11.681615,0.421306
4,2025-06-06,31.190001,31.190001,31.559000,28.350000,29.150000,3244100,NNE,4.655224,0.097081,57.122852,0.080707
5,2025-06-06,97.910004,97.910004,103.000000,95.000000,102.800003,6569700,RBRK,3.184065,-0.006696,164.768721,0.019051
6,2025-06-06,37.520000,37.520000,37.799999,37.009998,37.540001,4036300,AS,2.478203,0.003745,86.819697,0.028026
7,2025-06-06,35.480000,35.480000,35.580002,34.755001,35.070000,1715900,AHR,2.376423,0.014584,21.657313,0.107651
8,2025-06-06,10.135000,10.135000,10.470000,10.120000,10.470000,51200,SUPX,2.346065,-0.004420,8.637976,0.266389
9,2025-06-06,42.270000,42.270000,43.020000,42.126999,43.009998,460000,MRX,2.259220,-0.004006,30.146280,0.073449


In [300]:
stocke_df_20250606.sort_values(by='Sharpe', ascending=False).reset_index(drop=True).loc[:10].describe()

,Date,Adj Close,Close,High,Low,Open,Volume,Growth 252,Returns,Volatility (Yearly),Sharpe
count,11,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000
mean,2025-06-06 00:00:00,6.698909,6.698909,6.710182,6.665364,6.681182,74690.909091,1.576428,0.009757,1.963970,1.321075
min,2025-06-06 00:00:00,0.390000,0.390000,0.417000,0.376000,0.390000,0.000000,0.572687,-0.001916,0.352961,0.518080
25%,2025-06-06 00:00:00,1.565000,1.565000,1.585000,1.514500,1.545000,1300.000000,0.846078,0.000000,0.510133,0.856917
50%,2025-06-06 00:00:00,10.420000,10.420000,10.420000,10.420000,10.420000,7700.000000,1.045881,0.000000,0.827906,1.123493
75%,2025-06-06 00:00:00,10.710000,10.710000,10.720000,10.685000,10.700000,80850.000000,1.060633,0.007491,1.029412,1.788693
max,2025-06-06 00:00:00,10.890000,10.890000,10.890000,10.850000,10.850000,383400.000000,8.097413,0.079160,14.221305,2.835668
std,NaN,4.763891,4.763891,4.756570,4.771549,4.763455,123901.884203,2.171650,0.023359,4.074367,0.717352


### Question 3: Fixed Months Holding Strategy
What is the optimal number of months (1 to 12) to hold a newly IPO'd stock in order to maximize average growth?
(Assume you buy at the close of the first trading day and sell after a fixed number of trading days.)
* Using the IPOs data in the first 5 months of 2024 (data from question 2)

**Goal:**
Investigate whether holding an IPO stock for a fixed number of months after its first trading day produces better returns, using future growth columns.

In [182]:
trim_ipo_stock_df.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2024-05-31,NAKA,"Kindly MD, Inc.",NaN,14.18,NaN
1,2024-05-23,BOW,Bowhead Specialty Holdings Inc.,17.00,35.09,114.06
2,2024-05-17,HDL,Super Hi International Holding Ltd.,19.56,20.41,4.50
3,2024-05-17,RFAI,RF Acquisition Corp II,10.00,10.50,5.10
4,2024-05-15,JDZG,JIADE Limited,4.00,0.27,-92.63


In [184]:
# since in a month we have 21 trading days
future_growth_days = [x for x in range(21, 253, 21)]

for n_days in future_growth_days:
    trim_ipo_stock_df[f"Future Growth {n_days}d"] = trim_ipo_stock_df['IPO Price'].shift(-n_days) / trim_ipo_stock_df['IPO Price']

In [191]:
# Determine the first trading day (min_date) for each ticker.
first_trade_date = stock_df.groupby('Ticker')['Date'].min().reset_index()

first_trade_date.head()

,Ticker,Date
0,AHR,2024-02-07
1,ALAB,2024-03-20
2,ANRO,2024-02-02
3,AS,2024-02-01
4,AUNA,2024-03-22


In [197]:
# Inner join on both 'ticker' and 'date'
merged_df = pd.merge(first_trade_date, trim_ipo_stock_df, left_on=['Ticker', 'Date'], right_on=['Symbol', 'IPO Date'], how='inner')
merged_df = merged_df.drop(columns=['IPO Date', 'Symbol']) # avoid duplicated columns 

merged_df.shape

(59, 18)

In [201]:
merged_df.Ticker.nunique()

59

In [200]:
future_cols = [c for c in merged_df.columns if 'Future' in c]
merged_df[future_cols].describe()

,Future Growth 21d,Future Growth 42d,Future Growth 63d,Future Growth 84d,Future Growth 105d,Future Growth 126d,Future Growth 147d,Future Growth 168d,Future Growth 189d,Future Growth 210d,Future Growth 231d,Future Growth 252d
count,39.000000,25.000000,10.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,2.236829,1.859832,0.692785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,3.936827,1.960097,0.364190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.178571,0.204499,0.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.633333,0.500000,0.419048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.073620,1.000000,0.782353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,2.075000,2.600000,0.966447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,23.000000,7.500000,1.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


*  The average (mean) of 21 days future growth is 2.24, which is the highest among others. This means that holding an IPO stock for the first 21 days (1 month) produces better returns (median = 1.07).
*  This optimal month shows an uplift of `>1%` compared to all others.

### Question 4: Simple RSI-Based Trading Strategy
What is the total profit (in $thousands) you would have earned by investing `$1000` every time a stock was oversold (RSI < 25)?

**Goal:** Apply a simple rule-based trading strategy using the Relative Strength Index (RSI) technical indicator to identify oversold signals and calculate profits.

**Definition of Terms**

RSI measure how fast and how much a stock has been going up or down recently.

RSI is a number between 0 and 100.
 * RSI > 70 means price is going up very fast (overbought) and so it will fall soon.
 * RSI < 25 means price is going down very fast (oversold) and so it will rise soon.



In [230]:
['US'] * 5

['US', 'US', 'US', 'US', 'US']

In [235]:
# https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

# You're required to add EU_STOCKS and INDIA_STOCS
# https://companiesmarketcap.com/european-union/largest-companies-in-the-eu-by-market-cap/
EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

# https://companiesmarketcap.com/india/largest-companies-in-india-by-market-cap/
INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

ALL_TICKERS = US_STOCKS  + EU_STOCKS + INDIA_STOCKS


In [249]:
# zip the above together with the region they are from
# US_STOCKS = zip( US_STOCKS, ['US']*len(US_STOCKS) )
# EU_STOCKS = zip( EU_STOCKS, ['EU']*len(EU_STOCKS) )
# INDIA_STOCKS = zip( INDIA_STOCKS, ['INDIA']*len(INDIA_STOCKS) )

first_ticker = ALL_TICKERS[0]

stock_df = yf.download(tickers = first_ticker,
                     period = "max",
                     interval = "1d",
                     auto_adjust=False)
stock_df.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
stock_df.reset_index(inplace=True)
stock_df['Ticker'] = first_ticker

[*********************100%***********************]  1 of 1 completed


In [250]:
stock_df.head()

,Date,Adj Close,Close,High,Low,Open,Volume,Ticker
0,1986-03-13,0.059598,0.097222,0.101563,0.088542,0.088542,1031788800,MSFT
1,1986-03-14,0.061726,0.100694,0.102431,0.097222,0.097222,308160000,MSFT
2,1986-03-17,0.062791,0.102431,0.103299,0.100694,0.100694,133171200,MSFT
3,1986-03-18,0.061194,0.099826,0.103299,0.098958,0.102431,67766400,MSFT
4,1986-03-19,0.060130,0.098090,0.100694,0.097222,0.099826,47894400,MSFT


In [251]:
# get the remaining tickers
for ticker in tqdm(ALL_TICKERS[1:]):
    next_stock_df = yf.download(tickers = ticker,
                     period = "max",
                     interval = "1d",
                     auto_adjust=False)
    next_stock_df.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
    next_stock_df = next_stock_df.reset_index()
    next_stock_df['Ticker'] = ticker
    stock_df = pd.concat([stock_df, next_stock_df], ignore_index=True)

[*********************100%***********************]  1 of 1 completed                                                       | 0/32 [00:00<?, ?it/s]
[*********************100%***********************]  1 of 1 completed                                               | 1/32 [00:01<01:00,  1.95s/it]
[*********************100%***********************]  1 of 1 completed                                               | 2/32 [00:03<00:49,  1.67s/it]
[*********************100%***********************]  1 of 1 completed                                               | 3/32 [00:08<01:36,  3.31s/it]
[*********************100%***********************]  1 of 1 completed                                               | 4/32 [00:10<01:17,  2.76s/it]
[*********************100%***********************]  1 of 1 completed                                               | 5/32 [00:12<01:10,  2.61s/it]
[*********************100%***********************]  1 of 1 completed                                               | 6

In [254]:
# add ticker type column
def get_ticker_type(ticker:str, us_stocks_list, eu_stocks_list, india_stocks_list):
  if ticker in us_stocks_list:
    return 'US'
  elif ticker in eu_stocks_list:
    return 'EU'
  elif ticker in india_stocks_list:
    return 'INDIA'
  else:
    return 'ERROR'

stock_df['Ticker Type'] = stock_df.Ticker.apply(lambda x:get_ticker_type(x, US_STOCKS, EU_STOCKS, INDIA_STOCKS))

In [260]:
def compute_rsi(group):
    group['RSI'] = ta.RSI(group['Close'].values, timeperiod=14)
    return group

In [261]:
# Apply the function to each symbol group
stock_df = stock_df.groupby('Ticker').apply(compute_rsi).reset_index(drop=True)

C:\Users\DELL\AppData\Local\Temp\ipykernel_16984\1536603319.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stock_df = stock_df.groupby('Ticker').apply(compute_rsi).reset_index(drop=True)


In [267]:
# subset for the last 25 years
oversold_rsi_threshold = 25

selected_stock_df = stock_df[
    (stock_df['RSI'] < oversold_rsi_threshold) &
    (stock_df['Date'] >= '2000-01-01') &
    (stock_df['Date'] <= '2025-06-01')
]

In [275]:
# total number of trades
sum(selected_stock_df.groupby('Ticker')['Ticker'].count().values)

np.int64(1589)

* There were total number of `1589` trades from different companies.
* If we invest $1000 for each trade, using the 30-day forward return (growth_future_30d), our net earnings will be:

In [281]:
# calculating 30 future growth for each stock
selected_stock_df['30d Future Growth'] = selected_stock_df['Close'].shift(-30) / selected_stock_df['Close'] - 1

C:\Users\DELL\AppData\Local\Temp\ipykernel_16984\2830857919.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_stock_df['30d Future Growth'] = selected_stock_df['Close'].shift(-30) / selected_stock_df['Close'] - 1


In [301]:
net_income = 1000 * (selected_stock_df['30d Future Growth']).sum()
print(f"The net income as a result of investing $1K on each trade is: {round(net_income/1000, 2)}M")

The net income as a result of investing $1K on each trade is: 6953.21M


In [302]:
# net income investing $1k on any of the company
ticker_with_highest_growth = selected_stock_df.groupby('Ticker').apply(lambda group: 1000 * (group['30d Future Growth']).sum())
ticker_with_highest_growth_df = pd.DataFrame({'Tickers':ticker_with_highest_growth.index, 
                                              'Net Income':ticker_with_highest_growth.values})
ticker_with_highest_growth_df.sort_values(by='Net Income', ascending=False)

                                             

C:\Users\DELL\AppData\Local\Temp\ipykernel_16984\1322124344.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ticker_with_highest_growth = selected_stock_df.groupby('Ticker').apply(lambda group: 1000 * (group['30d Future Growth']).sum())


,Tickers,Net Income
0,AAPL,1.216744e+06
29,SIE.DE,6.641827e+05
22,NVDA,6.266543e+05
5,BHARTIARTL.NS,5.320527e+05
25,RELIANCE.NS,4.363244e+05
15,JPM,4.110155e+05
18,LT.NS,3.369259e+05
11,IBN,3.161221e+05
13,INFY,3.086965e+05
23,NVO,2.731621e+05


In [305]:
arr = np.arange(12) # 0 to 11 
print("\nOriginal array:", arr) 

# Reshape to 3 rows, 4 columns 
reshaped_arr = arr.reshape(3, 4) 
print("Reshaped to (3, 4):\n", reshaped_arr) 

# Reshape to 2 rows, infer columns 
inferred_shape = arr.reshape(2, -1) 
print("Reshaped to (2, -1):\n", inferred_shape) 

# Flatten an array back to 1D 
flat_arr = reshaped_arr.flatten() 
print("Flattened array:", flat_arr)



Original array: [ 0  1  2  3  4  5  6  7  8  9 10 11]
Reshaped to (3, 4):
 [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
Reshaped to (2, -1):
 [[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]]
Flattened array: [ 0  1  2  3  4  5  6  7  8  9 10 11]


In [1]:
5//2

2

In [2]:
5/2

2.5